# LN - Data PP - Stability and efficiency calculations

## Import libraries and data

In [92]:
import sqlite3
import numpy as np
import pandas as pd
import networkx as nx
import itertools
#import matplotlib.pyplot as plt
import time
import pickle

import os
import re
import sys
import io
from itertools import islice

from tqdm.notebook import trange, tqdm
from time import sleep

from dask_cloudprovider import FargateCluster
from dask.distributed import Client
import dask.array as da
import dask

import s3fs

import boto3


In [69]:


# Load Data

# Initiate s3 resource

session = boto3.session.Session()
s3 = session.resource('s3')


# Dataframe

decisions_load = s3.Object(bucket_name='ln-strategy-data', key='LN_channels.csv').get()
decisions_df=pd.read_csv(io.BytesIO(decisions_load['Body'].read()))

# Channel closures
closure_file = s3.Object(bucket_name='ln-strategy-data', key='channel_closures.p').get()
channel_closures = pickle.loads(closure_file['Body'].read())
    
    
# Channel openings 
opens_file = s3.Object(bucket_name='ln-strategy-data', key='channel_opens.p').get()
channel_opens = pickle.loads(opens_file['Body'].read())

    

# Create list with graph keys

#TODO: Save graphs as numpy array in single H5 file to reduce. Test if creating graphs takes longer than reading from S3

# graph_dir='./data/graph_snapshots' - For local tests
extraction_id=1585344554
graph_keys = [obj.key 
        for obj in s3.Bucket(name='ln-strategy-data').objects.all()
        if re.match(".*"+str(extraction_id)+"_connected/.*\.gpickle",obj.key)]





In [ ]:
# Test: extracted formats
print("---Sample graph keys---")
print(graph_keys[0])
print("---Sample channel opens---")
print(channel_opens[513675])
print("---Sample channel closures---")
print(channel_closures[592638])
print("----Sample blocks----")
print(blocks[1])


In [70]:
decisions_df.head()

,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,close_time,close_fee,last_update,close_type,close_htlc_count,close_balance_a,close_balance_b,dec_id,node0_id,node1_id
0,505149x622x0,505149,f6bc767df9148ebf76d2b9baf4eb46e3230712c2bf5a51...,bc1qjmg6ev344fenh3zhg0yjl6hyvxpxluw6x9nn2a5lv4...,592638.0,82cb2ea2a06c8c453d8b9ca08e17bbefe87225aa380b2d...,0250373555232cec757ea141273e75381c84cc3ab22f1e...,02ef61a252f9504a42fc264a28476f44cea0711a44b2da...,300000,2019-08-22 02:49:00,...,1.567276e+09,184.0,1.563172e+09,mutual,0.0,3570.0,296246.0,0,3098,1492
1,513675x2245x0,513675,4297b5fe9beeb701c67fd0f84861b22edbcafe5c25be67...,bc1qymmdt0vzhdjqyqw2cevrqppp6rrlg5j2l20yk72z6y...,594718.0,3f86d9427c750f37a963b5a329da8941520f5a6cdbfe02...,028aa5a991a2acf33da91674fe062219b640e5e57d77a4...,03fab7f8655169ea77d9691d4bd359e97782cb6177a6f7...,50000,2019-10-07 02:42:58,...,1.568401e+09,4410.0,1.552879e+09,unused,0.0,45590.0,0.0,1,5474,7365
2,513887x1177x0,513887,3b4cc434e62c1739e79171c7c1641bf9ac0e32d8530c68...,bc1q48l3h7sfdjaqat3sy98naltkwlujwefwnkfqxfm8fd...,NaN,NaN,02d97e94cfeedca2a3da47acb400bc6836e671b3cb3fc0...,03fab7f8655169ea77d9691d4bd359e97782cb6177a6f7...,50000,2020-02-14 03:15:22,...,NaN,NaN,1.581536e+09,NaN,NaN,NaN,NaN,2,2797,7365
3,513909x1248x0,513909,86311514680351b1e644276efd7704ba13be169cc1a272...,bc1q24kvd9wdjdhwgr54fmu7cu9xldmsjuwdsq2ph5fwnj...,NaN,NaN,02ad6fb8d693dc1e4569bcedefadf5f72a931ae027dc0f...,03fab7f8655169ea77d9691d4bd359e97782cb6177a6f7...,20000,2020-02-14 03:15:26,...,NaN,NaN,1.581464e+09,NaN,NaN,NaN,NaN,3,211,7365
4,513910x1814x1,513910,7f010765ce336d2be78c846844544e6a06ce2c59e7785f...,bc1qzh9xrpqvyse7fuanc8tl5e75qymq5fzk3deh78hs02...,NaN,NaN,02ad6fb8d693dc1e4569bcedefadf5f72a931ae027dc0f...,03fab7f8655169ea77d9691d4bd359e97782cb6177a6f7...,20000,2020-02-14 03:15:26,...,NaN,NaN,1.581466e+09,NaN,NaN,NaN,NaN,4,211,7365


In [ ]:
sys.getsizeof(decisions_df)

## Connection to AWS - Fargate Clusters

In [ ]:
cluster = FargateCluster(n_workers=20,scheduler_timeout='60 minutes',image='dsrincon/dask-graph:nx-scipy-v1')

In [71]:
cluster

In [95]:
client = Client(cluster)

/opt/conda/lib/python3.7/site-packages/distributed/client.py:1079: VersionMismatchWarning: Mismatched versions found

python
+---------------------------+---------------+
|                           | version       |
+---------------------------+---------------+
| client                    | 3.7.3.final.0 |
| scheduler                 | 3.7.4.final.0 |
| tcp://172.31.13.44:36759  | 3.7.4.final.0 |
| tcp://172.31.14.120:36793 | 3.7.4.final.0 |
| tcp://172.31.18.108:37605 | 3.7.4.final.0 |
| tcp://172.31.20.181:45127 | 3.7.4.final.0 |
| tcp://172.31.22.194:40509 | 3.7.4.final.0 |
| tcp://172.31.32.143:46031 | 3.7.4.final.0 |
| tcp://172.31.32.54:33291  | 3.7.4.final.0 |
| tcp://172.31.33.144:43449 | 3.7.4.final.0 |
| tcp://172.31.36.220:38213 | 3.7.4.final.0 |
| tcp://172.31.43.197:39039 | 3.7.4.final.0 |
| tcp://172.31.45.24:40141  | 3.7.4.final.0 |
| tcp://172.31.5.106:38861  | 3.7.4.final.0 |
| tcp://172.31.53.190:43377 | 3.7.4.final.0 |
| tcp://172.31.59.105:38475 | 3.7.4.final.0 |
|

## Extract Graphs for Analysis

In [98]:
# Lazy extract Graphs

# Function for lazy S3 extraction
def load_snapshots(key):
    session = boto3.session.Session()
    s3 = session.resource('s3')
    response = s3.Object(bucket_name='ln-strategy-data', key=key).get()
    G=pickle.loads(response['Body'].read())
    
    return G
    
# Script to create delayed array
graph_snapshots=[]
blocks=[]

for key in graph_keys[700:1000]: # Remove index for full range
    # Create block list from file_names
    block_i=int(key.split(".")[0].split("/")[-1]) 
    blocks.append(block_i)
    
    # Extract graphs
    G=dask.delayed(load_snapshots)(key)
    graph_snapshots.append(G)
   

In [ ]:
# Test Lazy Graph extract
graph_i=dask.compute(graph_snapshots[0])[0]
block=graph_i.graph['block']
print(type(block))

#graph_snapshots=dask.compute(*graph_snapshots)
#block=graph_snapshots[0].graph['block']
    
#print(len(graph_snapshots[5]))
#print(graph_snapshots[3].graph['block'])

# Delayed testing
#results = dask.compute(*futures)
#graphs=dask.compute(*graph_snapshots)

## Stability/Efficiency analysis by utility definition

In order to understand the potential motivations behind each decision we analyze each decission (opening or closure of a channel) independently from the perspective of each of the participants in the decission, which we'll call the node under analysis. For each decission we extract or compute the following information: 



## Betweeness


Betweenness centrality measures how central is a network to the flow of information in a network. In the case of the Lightning Network the higher the betweenness centrality of a node, the more transactions (messages) that are routed through it. In particular, we will use a measure of betweenness centrality defined in (Brandes and Fleischer 2005 - https://link.springer.com/chapter/10.1007/978-3-540-31856-9_44) that models infomation through a network, as electric current, efficiently and not only considering shortest path. This allows us to account for the fact that not all transactions travel through shortes path given that there are fee and capacity considerations.  

### Baseline betweeness

In [ ]:
# Test networkx (sequential computation)

snapshot_bet_list=[]

def bet_cent(g):
    #if len(g)>2:
    g_bet=nx.algorithms.centrality.approximate_current_flow_betweenness_centrality(g,weight='capacity',kmax=10000)
    block=g.graph['block']
    #else:
    #g_bet={}
    return (block,g_bet)
    

for g in graph_snapshots:
    block_bet_tuple=dask.delayed(bet_cent)(g)
    snapshot_bet_list.append(block_bet_tuple)

futures_bet = dask.persist(*snapshot_bet_list)

In [ ]:
start=time.time()
snapshot_bet_list = dask.compute(*futures_bet)
end=time.time()
print('Compute in seconds: {}'.format(end-start))

In [ ]:
# Define dictionary 
snapshot_bet={record[0]:record[1] for record in snapshot_bet_list}

In [ ]:
# Test results and size of betweeness in memory
# Create list with graph keys
print(sys.getsizeof(snapshot_bet))
n_items = take(10, snapshot_bet.items())
print(n_items)

In [97]:
blocks[:10]

[505149,
 506402,
 506847,
 508075,
 508090,
 508320,
 508400,
 508447,
 508503,
 508666]

### Pairwise stability 

- **Marginal betweenness (bet_mar_nodei)**: The % change between the betweenness centrality, for the node under analysis, given the graph from the previous block and the betweenness centrality of the resulting graph after enacting the decission (adding or removing a channel). Weighted current betweenness centrality is used for this measure.

In [102]:
#------STABILITY FOR OPENS----

# Function to calculate marginal betweenness centrality for all channel openings in snapshot

def bet_mar_open(input_tuple):
    
    block=input_tuple[0]
    G=input_tuple[1]
    block_opens=input_tuple[2]
    G_bets=input_tuple[3]
    #snapshot_bet=input_tuple[2] - Don't calculate difference
    
    # For each snapshot get all OPENS. 
    #block_opens=channel_opens[block]
    

    # For each open calculate marginal betweenness for each node in channel
    
    
    bet_mar_node0_dic_i={} #Dictionary to store marginal betweeness centrality for node0
    bet_mar_node1_dic_i={} #Dictionary to store marginal betweeness centrality for node0
    for open_edge in block_opens:
        
        # Extract info about channel
        channel_id=open_edge[2]['channel_id']
        node0=open_edge[0]
        node1=open_edge[1]
        edge_list=[open_edge]
        
        
        
        # Copy original graph
        g_mar=G.copy()   
        old_nodes=False
        
        
        
        # Retrieve betweenness for snapshot if nodes existed
        if (g_mar.has_node(node0)):
            node0_bet=G_bets[node0]
            old_nodes=True
            
        if (g_mar.has_node(node1)):
            node1_bet=G_bets[node1]
            old_nodes=True
            
            
        # Add edges and calculate betweeness if at least one of the nodes is in graph 
        
        if old_nodes: 
            g_mar.add_edges_from(edge_list)
            g_mar_bet=nx.algorithms.centrality.approximate_current_flow_betweenness_centrality(g_mar,weight='capacity',kmax=10000)
            node0_mar_bet=(g_mar_bet[node0]-node0_bet)
            node1_mar_bet=(g_mar_bet[node1]-node1_bet)
        else: # Make betweenness -10 if connection is outside of connected graph (both nodes are new)
            node0_mar_bet=-10
            node1_mar_bet=-10
        
        # Update dictionary - new betweenness
        bet_mar_node0_dic_i[channel_id]=node0_mar_bet
        bet_mar_node1_dic_i[channel_id]=node1_mar_bet
        
    
    return (bet_mar_node0_dic_i,bet_mar_node1_dic_i)
    

# Script to parallelize bet_mar_open

bet_mar_dicfut=[]
for i in range(len(graph_snapshots)):
    
    block=blocks[i]
    block_opens=channel_opens[block]
    #print((block_opens))
    g=graph_snapshots[i]
    g_bet=snapshot_bet[block]
    input_tuple=(block,g,block_opens,g_bet)
    output_tuple=dask.delayed(bet_mar_open)(input_tuple)
    bet_mar_dicfut.append(output_tuple)

futures_bet_mar = dask.persist(*bet_mar_dicfut)


In [103]:
# Run computation
start=time.time()
bet_mar_diclist = dask.compute(*futures_bet_mar)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(bet_mar_diclist)))


Compute in seconds: 15.94914698600769
Size in memory: 2448


In [104]:
# Test output
print(bet_mar_diclist[10])

({'526185x211x1': 0.05827316656294271, '526185x870x1': 0.02219399676110155}, {'526185x211x1': 4.185339268785951e-15, '526185x870x1': -0.00011371264886930049})


In [105]:
# Create single dictionaries for node0 and node1

bet_mar_node0_list=[t[0] for t in bet_mar_diclist]
bet_mar_node1_list=[t[1] for t in bet_mar_diclist]

bet_mar_node0_dic={}
for d in bet_mar_node0_list:
    bet_mar_node0_dic.update(d)
    
bet_mar_node1_dic={}
for d in bet_mar_node1_list:
    bet_mar_node1_dic.update(d)

# Test output
# print(bet_mar_node1_dic)

In [111]:
# Add to DataFrame

# Create empty columns
decisions_df['bet_mar_node0']=np.nan
decisions_df['bet_mar_node1']=np.nan

# Populate df with values
decisions_df['bet_mar_node0']=decisions_df['short_channel_id'].map(bet_mar_node0_dic)
decisions_df['bet_mar_node1']=decisions_df['short_channel_id'].map(bet_mar_node1_dic)

decisions_df_filter=decisions_df[decisions_df['bet_mar_node0'].notnull()]
print(len(decisions_df_filter))


348


**TODO**: Why is length of Dataframe longer thatn 



- **Actual % change in betweenness (bet_act_deltai)**: The % change between the betweenness centrality, for the node under analysis, given the graph from the previous block and the betweenness centrality of the resulting graph after enacting **all** the decissions (adding or removing a channels) in the current block. Weighted current betweenness centrality is used for this measure.

- **Marginal betweeness pairwise stability (bet_mar_pairstab)**: Evaluates if given the marginal graph that results from just enacting this decission is consistent with pairwise stability, from a betweenness perspective.

- **Actual betweeness pairwise stability (bet_act_pairstab)**: Evaluates if given the marginal graph that results from all the decisions in the block is consitend with pairwise stability, from a betweenness perspective. 




### Nash stability 

- **% Change with respect to not making decision (bet_binstat_deltai)**: The % change in betwewnness centrality, for the node under analysis, given the resulting graph after all of the decissions have been executed. 
- **Nash compatible - binary strategy (bet_binstat_nash)**: Returns true if given the other decissions enacted in the block not making decision would have NOT have resulted in higher betweenness centrality. This tells me if my strategy helped me be better off (took into account what others were doing)

(Optional approaches - Check for tracktability)
- **Nash compatible - close only strategy (bet_closestat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels would NOT have not resulted in higher betwneenness centrality. (NOTE: Check if there are combinatorial considerations, if so just look at closings up to x) 
- **Nash compatible - close/open (bet_allstat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels (with any node) or opening a channel with one of the round participants would NOT have not resulted in lower betwneenness centrality. (NOTE: To make it reasonable and constraint the strategy space only consider 'similar nodes' or with relationships in the past?).



### Efficiency
- **Average betweeness per block (bet_effic)**: Average betweenness centrality for all the nodes. 



## Connectivity

### Pairwise stability 

- **Marginal % change in connectivity (con_mar_deltai)**: The % change between the shortest path average, for the node under analysis, given the graph from the previous block and the shortest path average of the resulting graph after enacting the decission (adding or removing a channel). Weighted shortest path (_single_source_dijkstra_path_) is used for this measure.

- **Actual % change in connectivity (con_act_deltai)**: The % change between the shortest path average, for the node under analysis, given the graph from the previous block and the shortest path average of the resulting graph after enacting **all** the decissions (adding or removing a channels) in the current block. Weighted shortest path (_single_source_dijkstra_path_) is used for this measure.

- **Marginal connectivity pairwise stability (con_mar_pairstab)**: Evaluates if given the marginal graph that results from just enacting this decission is consistent with pairwise stability, from a connectivity perspective.

- **Actual connectivity pairwise stability (con_act_pairstab)**: Evaluates if given the marginal graph that results from all the decisions in the block is consitend with pairwise stability, from a connectivity perspective.  



### Nash stability 

- **% Change with respect to not making decision (con_binstat_deltai)**: The % change in shortest path average, for the node under analysis, given the resulting graph after all of the decissions have been executed. 
- **Nash compatible - binary strategy (con_binstat_nash)**: Returns true if given the other decissions enacted in the block not making decision would have NOT have resulted in higher shortest path average. NOTE: This indicates if the strategy selected made the node better off (took into account what others were doing)

(Optional approaches - Check for tracktability)
- **Nash compatible - close only strategy (con_closestat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels would NOT have not resulted in higher shortest path average. (NOTE: Check if there are combinatorial considerations, if so just look at closings up to x) 
- **Nash compatible - close/open (con_allstat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels (with any node) or opening a channel with one of the round participants would NOT have not resulted in lower shortest path average. (NOTE: To make it reasonable and constraint the strategy space only consider 'similar nodes' or with relationships in the past?).



### Efficiency
- **Average betweeness per block (bet_effic)**: Average shortest path average for all the nodes. 





## Utility Functions

In [93]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))